<a href="https://colab.research.google.com/github/szetong/WQU-Group-24/blob/master/WQU_Group_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Topic: Which Financial Ratios are best in Identifying Good buys**

---








## Part 1 : Empirical Analysis on Classification of Financial Ratios
1. Financial Ratio and Stock Price are obtained from MorningStar website (https://www.morningstar.com/)
2. Stock listed in China A50 Index will be used for this study
3. 10 years Annual Financial ratios will be used for this study


Import all the necessary librart for the analysis

In [0]:
import pandas as pd
import numpy as np
import io
from scipy.stats import kendalltau, pearsonr, spearmanr
from google.colab import files
from pandas.tseries.offsets import MonthEnd
import math
from IPython.display import display

Indicate the quantity of the stock to analyse for the same industry 

Example: **n = 13** (13 bank stock in China A50 Index)




In [0]:
n = 13

In [0]:
def Ratio_Dataframe(ratio_df,price_df):

  #REMOVE LAST COLUMN
  Ratio_df = ratio_df
  Ratio_df = Ratio_df.dropna(how='all')
  Ratio_df = Ratio_df.reset_index(drop= True)
  Ratio_df = Ratio_df.drop(['TTM'],axis=1)

  #SPLIT INTO FEW FINANCIAL RATIO DATAFRAME
  Ratio_1 = Ratio_df.iloc[:15, :]
  Ratio_Profit_1 = Ratio_df.iloc[16:27, :]
  Ratio_Profit_2 = Ratio_df.iloc[27:36, :]
  Ratio_CashFlow = Ratio_df.iloc[58:65, :]
  Ratio_Financial_Health = Ratio_df.iloc[64:86, :]
  Ratio_Liquidity = Ratio_df.iloc[86:91, :]
  Ratio_Efficiency = Ratio_df.iloc[92:, :]
  
  #GROWTH RATIO
  Growth_Rate_Revenue = Ratio_df.iloc[36:42, :]
  Growth_Rate_Revenue = Growth_Rate_Revenue.replace(to_replace="Year over Year", value= "Revenue_YOY", regex= True)
  Growth_Rate_Operating = Ratio_df.iloc[42:47, :]
  Growth_Rate_Operating = Growth_Rate_Operating.replace(to_replace="Year over Year", value= "Operating_YOY", regex= True)
  Growth_Rate_Income = Ratio_df.iloc[47:52, :]
  Growth_Rate_Income = Growth_Rate_Income.replace(to_replace="Year over Year", value= "Income_YOY", regex= True)
  Growth_Rate_EPS = Ratio_df.iloc[52:58, :]
  Growth_Rate_EPS = Growth_Rate_EPS.replace(to_replace="Year over Year", value= "EPS_YOY", regex= True)

  #PROFITABILITY RATIO
  Gross_Margin_Ratio = Ratio_Profit_2.loc[Ratio_Profit_2['Unnamed: 0']== 'Net Margin %']
  Return_On_Asset = Ratio_Profit_2.loc[Ratio_Profit_2['Unnamed: 0']== 'Return on Assets %']
  Return_On_Equity = Ratio_Profit_2.loc[Ratio_Profit_2['Unnamed: 0']== 'Return on Equity %']
  FCF_Sales = Ratio_Financial_Health.loc[Ratio_Financial_Health['Unnamed: 0']== 'Free Cash Flow/Sales %']
  FCF_Income = Ratio_Financial_Health.loc[Ratio_Financial_Health['Unnamed: 0']== 'Free Cash Flow/Net Income']

  #EFFICIENCY RATIO
  Payables_Period = Ratio_Efficiency.loc[Ratio_Efficiency['Unnamed: 0']== 'Payables Period']
  Cash_Conversion_Cycle = Ratio_Efficiency.loc[Ratio_Efficiency['Unnamed: 0']== 'Cash Conversion Cycle']
  Asset_Turnover = Ratio_Profit_2.loc[Ratio_Profit_2['Unnamed: 0']== 'Payables Period	']
  Fixed_Asset_Turnover = Ratio_Efficiency.loc[Ratio_Efficiency['Unnamed: 0']== 'Fixed Assets Turnover']

  #PRICE RATIO
  Dividend_Payout_Ratio = Ratio_1.loc[Ratio_1['Unnamed: 0']== 'Payout Ratio % *']
  Earning_Per_Share = Ratio_1.loc[Ratio_1['Unnamed: 0']== 'Earnings Per Share CNY']
  FCF_Per_Share = Ratio_1.loc[Ratio_1['Unnamed: 0']== 'Free Cash Flow Per Share * CNY']

  #LEVERAGE RATIO
  Debt_To_Equity = Ratio_Liquidity.loc[Ratio_Liquidity['Unnamed: 0']== 'Debt/Equity']
  Financial_Leverage = Ratio_Profit_2.loc[Ratio_Profit_2['Unnamed: 0']== 'Financial Leverage (Average)']
  Current_Ratio = Ratio_Liquidity.loc[Ratio_Liquidity['Unnamed: 0']== 'Current Ratio']
  Quick_Ratio = Ratio_Liquidity.loc[Ratio_Liquidity['Unnamed: 0']== 'Quick Ratio']

  #VALUATION RATIO
  Shares = Ratio_1.loc[Ratio_1['Unnamed: 0']== 'Shares Mil']
  BookValue_Per_Share = Ratio_1.loc[Ratio_1['Unnamed: 0']== 'Book Value Per Share * CNY']
  Revenue_Growth_YOY = Growth_Rate_Revenue.loc[Growth_Rate_Revenue['Unnamed: 0']== 'Revenue_YOY']
  Operation_Growth_YOY = Growth_Rate_Operating.loc[Growth_Rate_Operating['Unnamed: 0']== 'Operating_YOY']
  NetIncome_Growth_YOY = Growth_Rate_Income.loc[Growth_Rate_Income['Unnamed: 0']== 'Income_YOY']
  EPS_Growth_YOY = Growth_Rate_EPS.loc[Growth_Rate_EPS['Unnamed: 0']== 'EPS_YOY']

  #FILTER ONLY SELECTED FINANICAL RATIO
  Ratio_sum = pd.concat([Shares,Revenue_Growth_YOY,Operation_Growth_YOY,NetIncome_Growth_YOY,EPS_Growth_YOY,Return_On_Equity,Return_On_Asset,Gross_Margin_Ratio,Earning_Per_Share,BookValue_Per_Share,
                      FCF_Per_Share, Dividend_Payout_Ratio,Debt_To_Equity,Current_Ratio,Quick_Ratio,FCF_Sales,FCF_Income,Financial_Leverage,Asset_Turnover,Fixed_Asset_Turnover,
                      Cash_Conversion_Cycle,Payables_Period ], ignore_index=True)

  #MERGE YEAR END PRICE WITH RATIO
  Ratio_sum = Ratio_sum.rename(columns= {'Unnamed: 0': 'Description'})
  Ratio_sum = Ratio_sum.append({'Description' : 'Market Capital Mil'}, ignore_index = True)
  Ratio_sum = Ratio_sum.append({'Description' : 'Price/Earning Ratio'}, ignore_index = True)
  Ratio_sum = Ratio_sum.append({'Description' : 'Price/Book Ratio'}, ignore_index = True)
  Ratio_sum = Ratio_sum.append({'Description' : 'Price/FCF'}, ignore_index = True)
  Ratio_sum = Ratio_sum.append({'Description' : 'PEG Ratio'}, ignore_index = True)
  Ratio_sum = Ratio_sum.set_index('Description')
  Ratio_sum.columns = pd.to_datetime(Ratio_sum.columns) + MonthEnd(1)
  Ratio_sum.columns = Ratio_sum.columns.astype("datetime64[ns]")
  Price_df = price_df
  Price_df['Date']=pd.to_datetime(Price_df['Date'])
  Price_df = Price_df.drop(['Name','Volume'],axis=1)
  Price_df = Price_df.rename(columns= {'Price': 'Year End Price'})
  for column in Ratio_sum:
    Ratio_sum[column] = Ratio_sum[column].str.replace(",","").astype(float)
  comb = pd.merge_asof(left=Ratio_sum.T, left_index=True ,right=Price_df, right_on='Date')
  comb = comb.set_index('Date')
  comb = comb.T
  comb = comb.transform(pd.to_numeric, errors='coerce')
  comb.loc['Market Capital Mil'] = comb.loc['Year End Price'] * comb.loc['Shares Mil']
  comb.loc['Price/Earning Ratio'] = comb.loc['Year End Price'] / comb.loc['Earnings Per Share CNY']
  comb.loc['PEG Ratio'] = comb.loc['Price/Earning Ratio'] / comb.loc['EPS_YOY']
  comb.loc['Price/Book Ratio'] = comb.loc['Year End Price'] / comb.loc['Book Value Per Share * CNY']
  comb.loc['Price/FCF'] = comb.loc['Year End Price'] / comb.loc['Free Cash Flow Per Share * CNY']
  comb = comb.round(decimals=2)

  #REMOVE COLUMN IF STOCK PRICE ARE MISSING
  comb.loc['Year End Price'] = stock.loc['Year End Price'].replace(0, np.nan)
  comb = comb.dropna(axis=1)

  #OBTAIN CORRELATION FOR 3 DIFFERENT METHOD
  Ratio_correlation = comb
  #Ratio_correlation = Ratio_correlation.fillna(0)
  Ratio_correlation = Ratio_correlation.transpose()
  Ratio_correlation = Ratio_correlation.replace(np.inf,0)
  Ratio_correlation_P =  Ratio_correlation.corr(method='pearson')
  Ratio_correlation_K = Ratio_correlation.corr(method='kendall')
  Ratio_correlation_S = Ratio_correlation.corr(method='spearman')
  Summary_P = pd.DataFrame()
  Summary_P['Pearson Correlation'] = Ratio_correlation_P['Year End Price']
  Summary_K = pd.DataFrame()
  Summary_K['Kendal Correlation'] = Ratio_correlation_K['Year End Price']
  Summary_S = pd.DataFrame()
  Summary_S['Spearman Correlation'] = Ratio_correlation_S['Year End Price']
  
  #SAVE CORRELATION FILE TO LOCAL
  Name = pn
  Name = Name.replace('.csv','')
  
  Summary_P.to_csv(Name +'_Pearson.csv') 
  files.download(Name +'_Pearson.csv')
  Summary_K.to_csv(Name +'_Kendal.csv') 
  files.download(Name +'_Kendal.csv')
  Summary_S.to_csv(Name +'_Spearman.csv') 
  files.download(Name +'_Spearman.csv')

  #SAVE FINANCIAL RATIO SUMMARY FILE TO LOCAL
  comb.to_csv(Name +'_Summary.csv') 
  files.download(Name +'_Summary.csv')
    
  corr_combine = pd.DataFrame()
  #Name_2 = Name + 'Correlation'
  corr_combine[Name + ' Pearson'] = Summary_P['Pearson Correlation']
  corr_combine[Name + ' Kendall'] = Summary_K['Kendal Correlation']
  corr_combine[Name + ' Spearman'] = Summary_S['Spearman Correlation']

  return comb, corr_combine, Name

Upload the Financial Ratio files and Stock Price File that obtain from MorningStar 

(Example: Upload n times of both Financial Ratio Files and Stock Price File)

This code will download Correlation files and Summary file upon completion

In [0]:
stock_df = {}
corr = {}
name = {}
for i in range(n):
  print('\n Kindly upload Stock Key Financial Ratio File(CSV)')
  uploaded = files.upload()  
  for fn in uploaded.keys():
    stock_df[i] = pd.read_csv(fn,header=2)
    print('\n Kindly upload Stock Price File for "{name}" '.format(name=fn))
  
  uploaded_1 = files.upload()
  for pn in uploaded_1.keys():
    Price_raw = pd.read_csv(pn,index_col= False)
    stock_df[i], corr[i], name[i] = Ratio_Dataframe(stock_df[i],Price_raw)